In [1]:
# 라이브러리 가져오기
import pandas as pd # Series, DataFrame 사용
import numpy as np # 값을 생성하는데 사용

### 계층 색인

- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [2]:
# Series
# 인덱스에 다차원 리스트(아이템 2개)를 전달
# 다차원리스트[0] 상위계층
# 상위계층 작성시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
# 상위계층 리스트 개수 = 하위계층 리스트 개수
# 다차원리스트[1] : 하위계층
# 실습 --->
# 로우 인덱스 - 상위계층 : a, b, c, d
# 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)
s1 = pd.Series(np.arange(10), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'd'],
                                   [1, 2, 3, 1, 2, 1, 2, 3, 4, 1]])
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [3]:
# 인덱스 확인
s1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4),
            ('d', 1)],
           )

In [4]:
# 상위계층에 접근
# 계층 색인이 적용된 객체에 상위 인덱스에 접근 : 일반적인 Series 인덱싱으로 접근
s1['c']

1    5
2    6
3    7
4    8
dtype: int32

In [5]:
# 계층색인에 대하 ㄴ슬라이싱 : 마지막 인덱스도 포함('b': 'd'로 입력시 b, c, d로 조회)
s1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [6]:
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [7]:
# 하위계층에 접근
# a, b, c, d에서 하위계층 라벨번호 2인 데이터만 조회
# 데이터[상위계층명, 하위계층번호]
s1[:, 2]

a    1
b    4
c    6
dtype: int32

In [8]:
# 하위 계층에 슬라이싱 : loc메서드를 이용해서 슬라이싱
# 상위계층이 c이고 하위계층을 2~4까지 슬라이싱(.loc는 마지막 번호도 포함)
s1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [9]:
# unstack() : 최하위(기본동작) 에 있는 로우 계층을 컬럼으로 적용하여 위로 올림
# Series 객체를 DataFrame화 할 수 있음
# NaN : 기존에 없던 로우 계층에 대한 값
s1.unstack()

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


In [10]:
# stack() 메서드 : 컬럼에 있던 값을 가장 하위 인덱스로 내려서 재배열
# DataFrame을 다시 Series화 할 때 사용
s1.unstack().stack()

a  1    0.0
   2    1.0
   3    2.0
b  1    3.0
   2    4.0
c  1    5.0
   2    6.0
   3    7.0
   4    8.0
d  1    9.0
dtype: float64

In [11]:
# DataFrame 생성
# 구조 : 4 * 5
# 로우 인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 a, b)
# 컬럼 인덱스 : 상위(서울, 경기) / 하위(서울-강남, 잠실 / 경기-안양, 수원, 판교)
# 값 : 1씩 증가하는 20개
df = pd.DataFrame(np.arange(20).reshape(4, 5),
                 index=[[2017, 2017, 2018, 2018],
                       ['a', 'b', 'a', 'b']],
                 columns=[['서울', '서울', '경기', '경기', '경기'],
                         ['강남', '잠실', '안양', '수원', '판교']])
df

서울      경기        
        강남  잠실  안양  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [12]:
# 컬럼의 상위 계층 접근 : 일반적인 df컬럼 인덱싱 방식
# '서울' 데이터 조회
df['서울']

강남  잠실
2017 a   0   1
     b   5   6
2018 a  10  11
     b  15  16

In [13]:
# 컬럼이 이중으로 구조가 만들어져있으므로 인덱싱도 이중으로 해야 하나의 요소를 조회할 수 있음
df['서울']['강남']

2017  a     0
      b     5
2018  a    10
      b    15
Name: 강남, dtype: int32

In [14]:
# 튜플로 전달시 하나의 리스트로 2개 이상의 계층 처리 가능
df[('서울', '강남')]

2017  a     0
      b     5
2018  a    10
      b    15
Name: (서울, 강남), dtype: int32

In [15]:
# row의 상위 계층 접근
# 2017년 데이터를 조회해주세요.
df.loc[2017]

서울    경기      
  강남 잠실 안양 수원 판교
a  0  1  2  3  4
b  5  6  7  8  9

In [16]:
# 2017년도 전반기(a) 데이터 조회
df.loc[2017].loc['a']

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: a, dtype: int32

In [17]:
# 튜플로 전달
df.loc[(2017, 'a')]

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: (2017, a), dtype: int32

In [60]:
# 분당 ~ 수원까지 데이터 조회
# 주의점 : 데이터프레임 컬럼은 row 슬라이싱 결과에 대해서만 슬라이싱 적용 가능
# df['경기']['안양':'수원'] -> error
df['경기'].loc[:, "안양":"수원"] 

안양  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [19]:
# row의 최하위계층(a, b)를 컬럼으로 올리기 : unstack()
df.unstack()

서울              경기                    
      강남      잠실      안양      수원      판교    
       a   b   a   b   a   b   a   b   a   b
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [20]:
# 컬럼 최하위계층을 로우의 하위계층으로 재배열 : stack()
df.stack()

경기    서울
2017 a 강남   NaN   0.0
       수원   3.0   NaN
       안양   2.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     b 강남   NaN   5.0
       수원   8.0   NaN
       안양   7.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 a 강남   NaN  10.0
       수원  13.0   NaN
       안양  12.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     b 강남   NaN  15.0
       수원  18.0   NaN
       안양  17.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

In [21]:
# 계층의 인덱스번호 또는 라벨을 사용하여 상, 하위간 교환
# swaplevel(key1, key2, axis=0(기본값))
# axis가 0인 경우는 row의 상하위 개념이 뒤집힌다.
df.swaplevel(1, 0)

서울      경기        
        강남  잠실  안양  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [22]:
# swaplevel에 axis=1을 주면 계층이 아닌 컬럼에 변동이 생깁니다.
df.swaplevel(1, 0, axis=1)

강남  잠실  안양  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

### 객체 복사

- 할당기호(=) / 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성(얕은복사)
- obj.copy() : 원본과 독립적인 복사본 객체 생성(깊은복사)

In [23]:
# 모든 값이 1인 5 * 1 구조의 Series 생성
s1 = pd.Series(np.ones(5))
s1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [24]:
# 할당기호 = 로 복사 : 얕은복사
s2 = s1

In [25]:
# 할당기호로 복사한 객체의 아이템 수정
s2[0] = 10
s2

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [26]:
# = 을 이용한 대입은 원본객체도 변경됨
s1

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [27]:
# 원본객체의 아이템 수정
s1[1] = 3
s1

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [28]:
s2

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [29]:
# 슬라이싱으로 복사(일반 파이썬 -> 깊은복사, pandas->얕은복사로 취급됨)
s3 = s1[:]
s3

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [30]:
# 복사본 객체 아이템 수정
s3[2] = -9
s3

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [31]:
# s1과 s2 확인
print(s1)
print("------")
print(s2)

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64
------
0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64


In [32]:
# 원본 변경
s1[3] = 33
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [33]:
# 복사본 객체로 적용받음
print(s2)
print(s3)

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64


In [34]:
# .copy()를 이용한 깊은복사(복사)
s4 = s1.copy()

In [35]:
# s4 수정(s1 영향 X)
s4[4] = 44

In [36]:
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

In [37]:
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [38]:
# s1을 변경하는 경우에도 s4는 영향받지않음
s1[0] = 0

In [39]:
s1

0     0.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [40]:
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

### 정렬

- obj.sort_index() : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame, Series
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- obj.sort_values() : 값을 기준으로 정렬
    - DataFrame, Series
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [41]:
# Series생성
# 값과 인덱스라벨이 순서대로 들어가지 않은 Series
s1 = pd.Series([2, 3, 1, 7, 0], index=list('gacfd'))
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [42]:
# 인덱스 기준 오름차순으로 정렬
# 기본동작 : 오름차순 & 로우 인덱스
s1.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [43]:
# 인덱스 기준 내림차순으로 정렬
s1.sort_index(ascending=False)

g    2
f    7
d    0
c    1
a    3
dtype: int64

In [44]:
# 일시적 변경이므로 s1에는 영향을 주지 않음
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [45]:
# 값 기준 오름차순으로 정렬
s1.sort_values()

d    0
c    1
g    2
a    3
f    7
dtype: int64

In [46]:
# 값 기준 내림차순으로 정렬
s1.sort_values(ascending=False)

f    7
a    3
g    2
c    1
d    0
dtype: int64

In [47]:
# 원본에 바로 반영되지 않음
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [48]:
# DataFrame 생성
# 4 * 5, 무작위 정수(np.random.randint())
# 로우/컬럼 인덱스도 순서가 없는 값 지정
# 로우 'hcae', 컬럼은 'EAFCD'
df1 = pd.DataFrame(np.random.randint(20, size=(4, 5)), index=list('hcae'), columns=list('EAFCD'))
df1

,E,A,F,C,D
h,0,12,18,2,11
c,9,2,4,18,13
a,6,13,13,12,0
e,11,11,2,11,15


In [49]:
# 로우 인덱스 기준으로 오름차순 정렬(axis=0)
df1.sort_index()

,E,A,F,C,D
a,6,13,13,12,0
c,9,2,4,18,13
e,11,11,2,11,15
h,0,12,18,2,11


In [50]:
# 로우 인덱스 기준으로 내림차순 정렬(axis=0)
df1.sort_index(ascending=False)

,E,A,F,C,D
h,0,12,18,2,11
e,11,11,2,11,15
c,9,2,4,18,13
a,6,13,13,12,0


In [51]:
# 컬럼명 기준으로 오름차순 정렬(axis=1)
df1.sort_index(axis=1)

,A,C,D,E,F
h,12,2,11,0,18
c,2,18,13,9,4
a,13,12,0,6,13
e,11,11,15,11,2


In [52]:
# 컬럼명 기준으로 내림차순 정렬
df1.sort_index(axis=1, ascending=False)

,F,E,D,C,A
h,18,0,11,2,12
c,4,9,13,18,2
a,13,6,0,12,13
e,2,11,15,11,11


In [53]:
# 컬럼 기준으로 내림차순 정렬 후, 로우 기준으로 오름차순 정렬
df1.sort_index(axis=1, ascending=False).sort_index()

,F,E,D,C,A
a,13,6,0,12,13
c,4,9,13,18,2
e,2,11,15,11,11
h,18,0,11,2,12


In [54]:
# 원본에는 변경 없음
df1

,E,A,F,C,D
h,0,12,18,2,11
c,9,2,4,18,13
a,6,13,13,12,0
e,11,11,2,11,15


In [55]:
# 값 기준으로 정렬
# 컬럼 D의 값을 기준으로 오름차순 정렬
# sort_values(axis=0, by='기준컬럼명') : 기본동작(정렬결과를 로우에 반영)
df1.sort_values(by='D')

,E,A,F,C,D
a,6,13,13,12,0
h,0,12,18,2,11
c,9,2,4,18,13
e,11,11,2,11,15


In [56]:
# 컬럼 A를 기준으로 내림차순 정렬
df1.sort_values(by='A', ascending=False)

,E,A,F,C,D
a,6,13,13,12,0
h,0,12,18,2,11
e,11,11,2,11,15
c,9,2,4,18,13


In [57]:
# 인덱스라벨(row) c의 값을 오름차순으로 정렬
# 결과적으로 정렬되는 대상 : 컬럼
# 정렬의 기준 : row 라벨
df1.sort_values(axis=1, by='c')

,A,F,E,D,C
h,12,18,0,11,2
c,2,4,9,13,18
a,13,13,6,0,12
e,11,2,11,15,11


In [58]:
# 로우 'e'를 기준으로 값을 내림차순으로 정렬
df1.sort_values(axis=1, by='e', ascending=False)

,D,E,A,C,F
h,11,0,12,2,18
c,13,9,2,18,4
a,0,6,13,12,13
e,15,11,11,11,2
